In [0]:
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import functions as f
import datetime

In [0]:
spark = SparkSession.builder \
                    .appName("Breweries") \
                    .getOrCreate()

sc = spark.sparkContext

In [0]:
today = datetime.date.today()

In [0]:
df_silver = spark.read.format("delta").load("dbfs:/mnt/dados/silver/" + str(today))

In [0]:
df_grouped = df_silver.groupBy("Brewery_type", "State_province") \
                        .agg(f.count("*").alias("Quant_brewery"))

In [0]:
df_grouped.createOrReplaceTempView("brewery_view")

sql_query = """
SELECT Brewery_type, State_province, COUNT(*) as count
FROM brewery_view
GROUP BY Brewery_type, State_province
"""


In [0]:
df_view = spark.sql(sql_query)

In [0]:
gold_path = "dbfs:/mnt/dados/gold/"

In [0]:
try:
    df_view.write.format("delta") \
            .mode("append") \
            .save(gold_path)
    print("Data has been succesfully saved")

except Exception as error:
    print("An error has occured while writing data: {error}")